# Exhaustive Search for N=2

Implement fine-grained exhaustive search for N=2 configuration to find the global optimum.

In [1]:
import sys
import os
os.chdir('/home/code/experiments/004_exhaustive_n2')
sys.path.insert(0, '/home/code')

import numpy as np
import pandas as pd
import json
import time
from numba import njit, prange

# Import our custom modules
from code.tree_geometry import TX, TY, get_tree_vertices_numba, calculate_score
from code.overlap_check import polygons_overlap_numba, has_overlap
from code.utils import load_submission, save_submission, parse_submission

print("Modules imported successfully")

Modules imported successfully


In [2]:
# Load baseline and analyze N=2 configuration
baseline_df = pd.read_csv('/home/code/experiments/001_valid_baseline/submission.csv')
baseline_configs = parse_submission(baseline_df)

# Get baseline N=2
baseline_n2 = baseline_configs[2]
baseline_n2_score = calculate_score(baseline_n2)

print(f"Baseline N=2 configuration:")
for i, (x, y, angle) in enumerate(baseline_n2):
    print(f"  Tree {i}: x={x:.6f}, y={y:.6f}, angle={angle:.2f}°")
print(f"\nBaseline N=2 score: {baseline_n2_score:.6f}")

# Analyze the configuration
t1 = baseline_n2[0]
t2 = baseline_n2[1]
print(f"\nAngle difference: {abs(t1[2] - t2[2]):.2f}°")
print(f"Position difference: dx={t2[0]-t1[0]:.6f}, dy={t2[1]-t1[1]:.6f}")

Baseline N=2 configuration:
  Tree 0: x=0.154097, y=-0.038541, angle=203.63°
  Tree 1: x=-0.154097, y=-0.561459, angle=23.63°

Baseline N=2 score: 0.450779

Angle difference: 180.00°
Position difference: dx=-0.308194, dy=-0.522919


In [3]:
# Implement exhaustive search for N=2
@njit
def compute_bbox_side(rx1, ry1, rx2, ry2):
    """Compute bounding box side length for two sets of vertices."""
    min_x = min(rx1.min(), rx2.min())
    max_x = max(rx1.max(), rx2.max())
    min_y = min(ry1.min(), ry2.min())
    max_y = max(ry1.max(), ry2.max())
    return max(max_x - min_x, max_y - min_y)

@njit
def find_best_position_for_tree2(a1, a2, x_range, y_range, step):
    """Given angles a1 and a2, find the best position for tree2."""
    best_score = np.inf
    best_x, best_y = 0.0, 0.0
    
    # Get tree 1 vertices (at origin)
    rx1, ry1 = get_tree_vertices_numba(0.0, 0.0, a1)
    
    x_min, x_max = x_range
    y_min, y_max = y_range
    
    x2 = x_min
    while x2 <= x_max:
        y2 = y_min
        while y2 <= y_max:
            # Get tree 2 vertices
            rx2, ry2 = get_tree_vertices_numba(x2, y2, a2)
            
            # Check for overlap
            if not polygons_overlap_numba(rx1, ry1, rx2, ry2):
                side = compute_bbox_side(rx1, ry1, rx2, ry2)
                score = (side * side) / 2.0  # N=2
                if score < best_score:
                    best_score = score
                    best_x = x2
                    best_y = y2
            
            y2 += step
        x2 += step
    
    return best_score, best_x, best_y

print("Exhaustive search functions defined")

Exhaustive search functions defined


In [4]:
# Test the function with baseline angles
a1 = baseline_n2[0][2]
a2 = baseline_n2[1][2]

print(f"Testing with baseline angles: a1={a1:.2f}°, a2={a2:.2f}°")

# Coarse search first
start = time.time()
score, x2, y2 = find_best_position_for_tree2(a1, a2, (-1.5, 1.5), (-1.5, 1.5), 0.05)
elapsed = time.time() - start
print(f"Coarse search (step=0.05): score={score:.6f}, x2={x2:.4f}, y2={y2:.4f}, time={elapsed:.2f}s")

# Fine search around the best position
start = time.time()
score, x2, y2 = find_best_position_for_tree2(a1, a2, (x2-0.2, x2+0.2), (y2-0.2, y2+0.2), 0.01)
elapsed = time.time() - start
print(f"Fine search (step=0.01): score={score:.6f}, x2={x2:.4f}, y2={y2:.4f}, time={elapsed:.2f}s")

print(f"\nBaseline N=2 score: {baseline_n2_score:.6f}")
print(f"Difference: {baseline_n2_score - score:.6f}")

Testing with baseline angles: a1=203.63°, a2=23.63°


Coarse search (step=0.05): score=0.476860, x2=-0.3000, y2=-0.5500, time=1.19s
Fine search (step=0.01): score=0.457528, x2=-0.3100, y2=-0.5300, time=0.00s

Baseline N=2 score: 0.450779
Difference: -0.006749


In [5]:
# Full exhaustive search over all angle combinations
@njit
def exhaustive_n2_search(angle_step, position_step, x_range, y_range):
    """Exhaustive search over all angle combinations for N=2."""
    best_score = np.inf
    best_a1, best_a2 = 0.0, 0.0
    best_x2, best_y2 = 0.0, 0.0
    
    n_angles = int(360.0 / angle_step)
    
    for i in range(n_angles):
        a1 = i * angle_step
        for j in range(n_angles):
            a2 = j * angle_step
            
            score, x2, y2 = find_best_position_for_tree2(a1, a2, x_range, y_range, position_step)
            
            if score < best_score:
                best_score = score
                best_a1 = a1
                best_a2 = a2
                best_x2 = x2
                best_y2 = y2
    
    return best_score, best_a1, best_a2, best_x2, best_y2

print("Running exhaustive search with 5° angle step, 0.05 position step...")
start = time.time()
best_score, best_a1, best_a2, best_x2, best_y2 = exhaustive_n2_search(5.0, 0.05, (-1.5, 1.5), (-1.5, 1.5))
elapsed = time.time() - start

print(f"\nCoarse exhaustive search results:")
print(f"  Best score: {best_score:.6f}")
print(f"  Best angles: a1={best_a1:.1f}°, a2={best_a2:.1f}°")
print(f"  Best position: x2={best_x2:.4f}, y2={best_y2:.4f}")
print(f"  Time: {elapsed:.1f}s")
print(f"\nBaseline N=2 score: {baseline_n2_score:.6f}")
print(f"Improvement: {baseline_n2_score - best_score:.6f}")

Running exhaustive search with 5° angle step, 0.05 position step...



Coarse exhaustive search results:
  Best score: 0.476473
  Best angles: a1=65.0°, a2=245.0°
  Best position: x2=-0.5500, y2=-0.3000
  Time: 6.9s

Baseline N=2 score: 0.450779
Improvement: -0.025694


In [6]:
# Refine around the best solution found
print(f"\nRefining around best solution (a1={best_a1:.1f}°, a2={best_a2:.1f}°)...")

# Fine angle search around best angles
fine_best_score = best_score
fine_best_a1, fine_best_a2 = best_a1, best_a2
fine_best_x2, fine_best_y2 = best_x2, best_y2

start = time.time()
for da1 in np.arange(-5, 6, 1.0):  # ±5° around best_a1
    for da2 in np.arange(-5, 6, 1.0):  # ±5° around best_a2
        a1 = (best_a1 + da1) % 360
        a2 = (best_a2 + da2) % 360
        
        # Fine position search
        score, x2, y2 = find_best_position_for_tree2(a1, a2, (-1.5, 1.5), (-1.5, 1.5), 0.02)
        
        if score < fine_best_score:
            fine_best_score = score
            fine_best_a1, fine_best_a2 = a1, a2
            fine_best_x2, fine_best_y2 = x2, y2

elapsed = time.time() - start
print(f"Fine search completed in {elapsed:.1f}s")
print(f"\nRefined results:")
print(f"  Best score: {fine_best_score:.6f}")
print(f"  Best angles: a1={fine_best_a1:.1f}°, a2={fine_best_a2:.1f}°")
print(f"  Best position: x2={fine_best_x2:.4f}, y2={fine_best_y2:.4f}")
print(f"\nBaseline N=2 score: {baseline_n2_score:.6f}")
print(f"Improvement: {baseline_n2_score - fine_best_score:.6f}")


Refining around best solution (a1=65.0°, a2=245.0°)...


Fine search completed in 1.0s

Refined results:
  Best score: 0.464989
  Best angles: a1=67.0°, a2=247.0°
  Best position: x2=-0.5200, y2=-0.3200

Baseline N=2 score: 0.450779
Improvement: -0.014210


In [7]:
# The exhaustive search found worse scores than baseline!
# Let's search specifically around the baseline angles with very fine resolution

print("Searching around baseline angles with very fine resolution...")
print(f"Baseline angles: a1={baseline_n2[0][2]:.2f}°, a2={baseline_n2[1][2]:.2f}°")

baseline_a1 = baseline_n2[0][2]
baseline_a2 = baseline_n2[1][2]

# Very fine search around baseline
best_score = baseline_n2_score
best_a1, best_a2 = baseline_a1, baseline_a2
best_x2, best_y2 = baseline_n2[1][0] - baseline_n2[0][0], baseline_n2[1][1] - baseline_n2[0][1]

start = time.time()
for da1 in np.arange(-10, 11, 0.5):  # ±10° around baseline_a1
    for da2 in np.arange(-10, 11, 0.5):  # ±10° around baseline_a2
        a1 = (baseline_a1 + da1) % 360
        a2 = (baseline_a2 + da2) % 360
        
        # Fine position search
        score, x2, y2 = find_best_position_for_tree2(a1, a2, (-1.0, 1.0), (-1.0, 1.0), 0.01)
        
        if score < best_score - 1e-9:
            best_score = score
            best_a1, best_a2 = a1, a2
            best_x2, best_y2 = x2, y2
            print(f"  Found better: score={score:.6f}, a1={a1:.2f}°, a2={a2:.2f}°")

elapsed = time.time() - start
print(f"\nSearch completed in {elapsed:.1f}s")
print(f"\nBest results:")
print(f"  Best score: {best_score:.6f}")
print(f"  Best angles: a1={best_a1:.2f}°, a2={best_a2:.2f}°")
print(f"  Best position: x2={best_x2:.6f}, y2={best_y2:.6f}")
print(f"\nBaseline N=2 score: {baseline_n2_score:.6f}")
print(f"Improvement: {baseline_n2_score - best_score:.6f}")

Searching around baseline angles with very fine resolution...
Baseline angles: a1=203.63°, a2=23.63°



Search completed in 29.2s

Best results:
  Best score: 0.450779
  Best angles: a1=203.63°, a2=23.63°
  Best position: x2=-0.308194, y2=-0.522919

Baseline N=2 score: 0.450779
Improvement: 0.000000


In [ ]:
# The baseline N=2 is already optimal!
# Update variables for the rest of the notebook
very_fine_best_score = best_score
very_fine_best_a1 = best_a1
very_fine_best_a2 = best_a2
very_fine_best_x2 = best_x2
very_fine_best_y2 = best_y2

print(f"\\nConclusion: Baseline N=2 is already optimal (or very close to it)")
print(f"No improvement found after exhaustive search")

In [ ]:
# Create the best N=2 configuration and validate
best_n2_config = [
    (0.0, 0.0, very_fine_best_a1),
    (very_fine_best_x2, very_fine_best_y2, very_fine_best_a2)
]

# Validate no overlap
if has_overlap(best_n2_config):
    print("❌ ERROR: Best configuration has overlaps!")
else:
    print("✅ Best configuration is valid (no overlaps)")

# Verify score
verified_score = calculate_score(best_n2_config)
print(f"Verified score: {verified_score:.6f}")

# Compare to baseline
improvement = baseline_n2_score - verified_score
if improvement > 1e-6:
    print(f"\n✅ IMPROVEMENT FOUND: {improvement:.6f} ({100*improvement/baseline_n2_score:.2f}%)")
else:
    print(f"\n❌ No improvement over baseline")

In [ ]:
# Create full submission with updated N=2 (if improved)
improved_configs = {n: list(baseline_configs[n]) for n in range(1, 201)}

if improvement > 1e-6:
    improved_configs[2] = best_n2_config
    print(f"Updated N=2 configuration")
else:
    print(f"Keeping baseline N=2 configuration")

# Calculate total score
new_total = sum(calculate_score(improved_configs[n]) for n in range(1, 201))
baseline_total = sum(calculate_score(baseline_configs[n]) for n in range(1, 201))

print(f"\nBaseline total: {baseline_total:.6f}")
print(f"New total: {new_total:.6f}")
print(f"Total improvement: {baseline_total - new_total:.6f}")

In [ ]:
# Save submission
save_submission(improved_configs, 'submission.csv')
print("Saved submission.csv")

# Save metrics
metrics = {
    'cv_score': new_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - new_total,
    'n2_baseline_score': baseline_n2_score,
    'n2_new_score': verified_score,
    'n2_improvement': improvement,
    'best_config': {
        'a1': very_fine_best_a1,
        'a2': very_fine_best_a2,
        'x2': very_fine_best_x2,
        'y2': very_fine_best_y2
    },
    'notes': 'Exhaustive search for N=2 with hierarchical refinement'
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nFinal CV Score: {new_total:.6f}")

In [ ]:
# Copy to submission folder
import shutil
shutil.copy('submission.csv', '/home/submission/submission.csv')
print("Copied submission to /home/submission/")